In [ ]:
pip install pandas scikit-learn numpy


In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the data from the CSV file

def load_data():
    file_path = "//content/Gabor_with_labels_andinfo (1).csv"  # Specify the file path directly in the function
    return pd.read_csv(file_path)

# Function to apply a sliding window over the features
def sliding_window_feature_selection(data, target_column, window_size=3):
    # Separate the features and target
    X = data.drop(columns=[target_column, 'ImageName'])  # Drop 'ImageName' as it's not a feature
    y = data[target_column]

    # Initialize variables to track best features and best accuracy
    best_accuracy = 0
    best_features = None

    # Perform sliding window over the features
    for start in range(0, X.shape[1] - window_size + 1):
        # Select window of features
        selected_features = X.iloc[:, start:start + window_size]

        # Split into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(selected_features, y, test_size=0.3, random_state=42)

        # Train a classifier (Random Forest in this case)
        clf = RandomForestClassifier(random_state=42)
        clf.fit(X_train, y_train)

        # Predict and calculate accuracy
        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        # Update best features if current window improves accuracy
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_features = selected_features.columns

    print(f"Best accuracy: {best_accuracy}")
    print(f"Selected features: {best_features}")
    return best_features

# Example usage
csv_file = '/content/Gabor_with_labels_andinfo (1).csv'  # Path to your CSV file
target_column = "Label"  # 'Label' is the target column for classification

# Load data
data = load_data()

# Apply sliding window feature selection
best_features = sliding_window_feature_selection(data, target_column, window_size=3)



Best accuracy: 1.0
Selected features: Index(['GaborEnergy_1_1_Window1', 'GaborEnergy_1_1_Window2',
       'GaborEnergy_1_1_Window3'],
      dtype='object')


In [2]:
pip install hmmlearn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 3.5 MB/s eta 0:00:00


In [3]:
import pandas as pd

# Load the CSV file (replace with your correct file path)
file_path = '/content/Gabor_with_labels_andinfo (1).csv'  # Adjust the file path accordingly
data = pd.read_csv(file_path)

# Inspect the first few rows of the data
print(data.head())

# List all columns to identify which columns are features and which column is the label
print(data.columns)


                  ImageName  Label  GaborEnergy_1_1_Window1  \
0  MaI12_Page100_line_1.jpg  color             2.918977e+09   
1  MaI12_Page100_line_2.jpg  color             5.055518e+09   
2  MaI12_Page100_line_3.jpg  color             4.993950e+09   
3  MaI12_Page100_line_4.jpg  color             3.740395e+09   
4  MaI12_Page100_line_5.jpg  color             2.482349e+09   

   GaborEnergy_1_1_Window2  GaborEnergy_1_1_Window3  GaborEnergy_1_1_Window4  \
0             3.180004e+09             3.141088e+09             3.133615e+09   
1             5.121529e+09             5.059220e+09             4.972092e+09   
2             4.954889e+09             4.614656e+09             4.559201e+09   
3             3.820795e+09             3.559340e+09             3.590960e+09   
4             3.347111e+09             3.227321e+09             2.759642e+09   

   GaborEnergy_1_1_Window5  GaborEnergy_1_1_Window6  GaborEnergy_1_1_Window7  \
0             2.809958e+09             2.966913e+09         

In [13]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from hmmlearn import hmm  # Hidden Markov Model implementation from hmmlearn

# Load the sliding window features file
def load_data():
    file_path = "/content/Gabor_with_labels_andinfo (1).csv"  # Adjust the path as needed
    return pd.read_csv(file_path)

# Load the ground truth table from the Excel file
def load_ground_truth():
    gt_file_path = "//content/line_gt 2.xlsx"  # Adjust the path as needed
    return pd.read_excel(gt_file_path)

# Function to apply a sliding window over the features (as previously implemented)
def sliding_window_feature_selection(data, target_column, window_size=3):
    X = data.drop(columns=[target_column, 'ImageName'])  # Drop 'ImageName' as it's not a feature
    y = data[target_column]

    best_accuracy = 0
    best_features = None

    for start in range(0, X.shape[1] - window_size + 1):
        selected_features = X.iloc[:, start:start + window_size]
        X_train, X_test, y_train, y_test = train_test_split(selected_features, y, test_size=0.3, random_state=42)

        clf = RandomForestClassifier(random_state=42)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_features = selected_features.columns

    print(f"Best accuracy: {best_accuracy}")
    print(f"Selected features: {best_features}")
    return best_features

# Baum-Welch (B-W) Algorithm for training HMM
def train_hmm_with_baum_welch(features, ground_truth):
    # Assume we have some observations (features) and corresponding ground truth
    X = np.array(features)  # Features from sliding window selection
    y = np.array(ground_truth)  # Ground truth sequence

    # HMM parameters (initialize arbitrarily)
    n_states = 3  # Example: number of hidden states in the HMM
    model = hmm.GaussianHMM(n_components=n_states, covariance_type="diag", n_iter=100)

    # Fit the HMM to the features (training using Baum-Welch)
    model.fit(X)

    # Output the HMM parameters (transitions, means, etc.)
    print("Transition matrix:", model.transmat_)
    print("Means of the Gaussian emissions:", model.means_)
    print("Covariances of the Gaussian emissions:", model.covars_)

    return model

# Example usage
csv_file = '/content/Gabor_with_labels_andinfo (1).csv'  # Path to your CSV file
target_column = "Label"  # Assuming 'Label' is the target column for classification
ground_truth_file = '/content/line_gt 2.xlsx'  # Path to your ground truth file

# Load data
data = load_data()

# Apply sliding window feature selection
best_features = sliding_window_feature_selection(data, target_column, window_size=3)

# Load ground truth
ground_truth = load_ground_truth()

# Train the HMM using the selected features and ground truth
hmm_model = train_hmm_with_baum_welch(data[best_features], ground_truth['gt'])


Best accuracy: 1.0
Selected features: Index(['GaborEnergy_1_1_Window1', 'GaborEnergy_1_1_Window2',
       'GaborEnergy_1_1_Window3'],
      dtype='object')
Transition matrix: [[0.90852379 0.03067388 0.06080232]
 [0.18208592 0.72698792 0.09092616]
 [0.03071226 0.06031595 0.9089718 ]]
Means of the Gaussian emissions: [[4.13304513e+09 4.07126720e+09 4.05344415e+09]
 [1.96868544e+09 2.10751762e+09 2.41700761e+09]
 [6.48991136e+09 6.32824818e+09 6.27438222e+09]]
Covariances of the Gaussian emissions: [[[4.88230391e+17 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 3.83553620e+17 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 3.03585634e+17]]

 [[1.02659575e+17 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 1.29967403e+17 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 1.46559717e+17]]

 [[7.07511014e+17 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 6.23148448e+17 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 5.58871772e+17]]]


In [14]:
import pandas as pd
import numpy as np
from hmmlearn import hmm

# Load the feature data
def load_feature_data():
    feature_file_path = "//content/Gabor_with_labels_andinfo (1).csv"
    return pd.read_csv(feature_file_path)

# Load the ground truth data
def load_ground_truth_data():
    ground_truth_file_path = "//content/line_gt 2.xlsx"
    return pd.read_excel(ground_truth_file_path)

# Function to apply a sliding window over the features
def sliding_window_feature_selection(data, target_column, window_size=3):
    # Separate the features and target
    X = data.drop(columns=[target_column, 'ImageName'])  # Drop 'ImageName' as it's not a feature
    y = data[target_column]

    # Initialize variables to track best features and best accuracy
    best_accuracy = 0
    best_features = None

    # Perform sliding window over the features
    for start in range(0, X.shape[1] - window_size + 1):
        # Select window of features
        selected_features = X.iloc[:, start:start + window_size]

        # Split into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(selected_features, y, test_size=0.3, random_state=42)

        # Train a classifier (Random Forest in this case)
        clf = RandomForestClassifier(random_state=42)
        clf.fit(X_train, y_train)

        # Predict and calculate accuracy
        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        # Update best features if current window improves accuracy
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_features = selected_features.columns

    print(f"Best accuracy: {best_accuracy}")
    print(f"Selected features: {best_features}")
    return best_features

# Function to apply the Baum-Welch algorithm (HMM training)
def train_hmm(features, ground_truth, n_states=5, n_iter=10):
    # Initialize the HMM model
    model = hmm.GaussianHMM(n_components=n_states, covariance_type="diag", n_iter=n_iter)

    # Fit the model using the ground truth as observations and features as the input data
    model.fit(features)

    # Predict the hidden states (inferred sequence)
    hidden_states = model.predict(features)

    print("Hidden States Sequence:", hidden_states)
    return hidden_states

# Load the data
data = load_feature_data()
ground_truth_data = load_ground_truth_data()

# Apply sliding window feature selection
best_features = sliding_window_feature_selection(data, target_column="Label", window_size=3)

# Use the selected features for the HMM
selected_features_data = data[best_features]

# Train HMM using the Baum-Welch algorithm and the ground truth
hidden_states = train_hmm(selected_features_data, ground_truth_data, n_states=5, n_iter=100)


Best accuracy: 1.0
Selected features: Index(['GaborEnergy_1_1_Window1', 'GaborEnergy_1_1_Window2',
       'GaborEnergy_1_1_Window3'],
      dtype='object')
Hidden States Sequence: [1 0 0 1 1 1 4 4 0 2 1 3 3 4 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 2 2 2 2 2
 2 2 2 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 3 3 4 4 4 3 3 4 4 2 1 1 1 0 1
 1 1 0 0]


In [15]:
import pandas as pd

# Load the Excel file containing the ground truth (Malayalam characters)
def load_ground_truth(file_path):
    # Assuming the ground truth data (characters) is in the first column of the sheet
    df = pd.read_excel(file_path, sheet_name=0)

    # Convert the column with Malayalam characters to a list
    malayalam_characters = df.iloc[:, 0].tolist()  # Assuming the characters are in the first column
    return malayalam_characters

# Convert hidden states to Malayalam characters using the ground truth list
def convert_states_to_characters(hidden_states, malayalam_characters):
    malayalam_sequence = [malayalam_characters[state] for state in hidden_states]
    return ''.join(malayalam_sequence)

# Load the ground truth Malayalam characters from the Excel file
excel_file_path = '/content/line_gt 2.xlsx'  # Path to the provided Excel file
malayalam_characters = load_ground_truth(excel_file_path)

# Hidden states output from the HMM
hidden_states = [1, 0, 0, 1, 1, 1, 4, 4, 0, 2, 1, 3, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2,
                 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 3, 4, 4, 4, 3, 3, 4, 4, 2, 1, 1, 1, 0, 1,
                 1, 1, 0, 0]

# Convert the hidden states to the corresponding Malayalam characters using the ground truth
malayalam_sequence = convert_states_to_characters(hidden_states, malayalam_characters)

# Print the Malayalam character sequence
print("Malayalam Character Sequence:", malayalam_sequence)


Malayalam Character Sequence: മാളെന്നരാജാവിനഅണ കലിസ്വർഗ്ഗ സന്ദേഹ പ്രാപ്യ പ ചെരമാന്‍ പെരുമാളെന്ന രാജാ അവരൊധാ കഴിച്ചു കൊള്ളുവാൻ തക്കവണ്ണം എച്ചും നീരും കൊടുത്തു ചെരമാന്‍ പെരു അവരൊധാ കഴിച്ചു കൊള്ളുവാൻ തക്കവണ്ണം എച്ചും നീരും കൊടുത്തു ചെരമാന്‍ പെരുമാളെന്നരാജാവിനഅണ കലിസ്വർഗ്ഗ സന്ദേഹ പ്രാപ്യ പ ചെരമാന്‍ പെരുമാളെന്ന രാജാമാളെന്നരാജാവിനഅണ കലിസ്വർഗ്ഗ സന്ദേഹ പ്രാപ്യ പ ചെരമാന്‍ പെരുമാളെന്ന രാജാമാളെന്നരാജാവിനഅണ കലിസ്വർഗ്ഗ സന്ദേഹ പ്രാപ്യ പ ചെരമാന്‍ പെരുമാളെന്ന രാജാച്ച കൃഷ്ണരായ രാ മലയാളം അടക്കുവാൻ പടക്കൂട്ടുകഎല്ലോ ചൈക്കാത്ത അതിച്ച കൃഷ്ണരായ രാ മലയാളം അടക്കുവാൻ പടക്കൂട്ടുകഎല്ലോ ചൈക്കാത്ത അതി അവരൊധാ കഴിച്ചു കൊള്ളുവാൻ തക്കവണ്ണം എച്ചും നീരും കൊടുത്തു ചെരമാന്‍ പെരുവിൻറെ ഗുണാധിക്യംമുപ്പത്തിയാറുവർഷം കാലം പാണു മാളെന്നരാജാവിനഅണ കലിസ്വർഗ്ഗ സന്ദേഹ പ്രാപ്യ പ ചെരമാന്‍ പെരുമാളെന്ന രാജാബ്രാഹ്മണരപരദേശത്ത ചെന്നതുമില്ലപെരുമാളെ കണ്ടതുമില്ല എന്നുകൽപ്പിബ്രാഹ്മണരപരദേശത്ത ചെന്നതുമില്ലപെരുമാളെ കണ്ടതുമില്ല എന്നുകൽപ്പിച്ച കൃഷ്ണരായ രാ മലയാളം അടക്കുവാൻ പടക്കൂട്ടുകഎല്ലോ ചൈക്കാത്ത അതി അവരൊധാ കഴിച്ചു കൊള്ളുവാൻ തക്കവണ്ണം എച്ചും നീരും ക

In [18]:
import pandas as pd
import numpy as np

# Function to compute Levenshtein distance
def levenshtein_distance(seq1, seq2):
    """ Compute the Levenshtein distance between two sequences. """
    len1, len2 = len(seq1), len(seq2)

    # Initialize a 2D array to store distances
    dp = np.zeros((len1 + 1, len2 + 1), dtype=int)

    # Initialize base cases
    for i in range(len1 + 1):
        dp[i][0] = i
    for j in range(len2 + 1):
        dp[0][j] = j

    # Compute the distances
    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            if seq1[i - 1] == seq2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]  # No change needed
            else:
                dp[i][j] = min(dp[i - 1][j] + 1,     # Deletion
                               dp[i][j - 1] + 1,     # Insertion
                               dp[i - 1][j - 1] + 1) # Substitution

    return dp[len1][len2]

# Function to compute Character Error Rate (CER)
def compute_cer(predicted_sequence, ground_truth_sequence):
    """ Computes the Character Error Rate (CER) between two sequences. """
    distance = levenshtein_distance(predicted_sequence, ground_truth_sequence)
    cer = distance / len(ground_truth_sequence)
    return cer, distance

# Load the ground truth Malayalam character sequence (from Excel)
def load_ground_truth(file_path):
    """ Load ground truth sequence of Malayalam characters from an Excel file. """
    df = pd.read_excel(file_path, sheet_name=0)
    ground_truth_characters = df.iloc[:, 0].tolist()

    # Handle non-string (NaN or float) values
    ground_truth_characters = [str(char) if pd.notna(char) else '' for char in ground_truth_characters]

    return ''.join(ground_truth_characters)

# Example sequences (replace with your actual sequences)
excel_file_path = '/content/line_gt 2.xlsx'  # Path to the provided Excel file
ground_truth_sequence = load_ground_truth(excel_file_path)

# Predicted Malayalam character sequence (replace with actual output from the previous step)
predicted_sequence = "your_predicted_malayalam_sequence_here"  # Replace this with actual prediction

# Compute CER
cer, edit_distance = compute_cer(predicted_sequence, ground_truth_sequence)

# Output the results
print(f"Ground Truth Sequence: {ground_truth_sequence}")
print(f"Predicted Sequence: {predicted_sequence}")
print(f"Levenshtein Distance: {edit_distance}")
print(f"Character Error Rate (CER): {cer}")


Ground Truth Sequence:  അവരൊധാ കഴിച്ചു കൊള്ളുവാൻ തക്കവണ്ണം എച്ചും നീരും കൊടുത്തു ചെരമാന്‍ പെരുമാളെന്നരാജാവിനഅണ കലിസ്വർഗ്ഗ സന്ദേഹ പ്രാപ്യ പ ചെരമാന്‍ പെരുമാളെന്ന രാജാവിൻറെ ഗുണാധിക്യംമുപ്പത്തിയാറുവർഷം കാലം പാണു ബ്രാഹ്മണരപരദേശത്ത ചെന്നതുമില്ലപെരുമാളെ കണ്ടതുമില്ല എന്നുകൽപ്പിച്ച കൃഷ്ണരായ രാ മലയാളം അടക്കുവാൻ പടക്കൂട്ടുകഎല്ലോ ചൈക്കാത്ത അതിൻറെ ശേഷം അറുപത്തനാല ഗ്രാമത്തിൽ ബ്രാഹ്മണരുംചേരമാൻപെരുമാളെന്ന രാജാവുംതൃക്കാരിയൂർ പൊന്മാടത്തിൽകീഴ അടിയന്തരമീ രുന്ന അടിയന്തര സഭ യിന്ന നിരൂപിച്ചു പതിനെ ഴ നാട്ടിലുള്ള നാട്ടിലുള്ള പുരുഷാരത്തെ എത്തിച്ചു ദിഗ് വിജയം ഉണ്ടായിട്ട ആരുള്ളു എന്ന അമ്പെഷിപ്പത്തിന്റെ ശേഷം പൂന്തുറെ ഉണ്ടമാന വിക്കീരൻ ലിപ്പാൻ രാജാവാവെണമെന്ന  കൽപ്പിച്ച ബ്രാഹ്മണർ  പരശത്തെ പരദെശത്തെ  ചെന്ന  കൃഷ്ണ രാ യരുമായി ക്കണ്ട  പന്തീരാണ്ട   വാഴുവാൻ  തക്കവണ്ണം   ആദിരാജാ  പെരുമാളെയും വൊണ്ടി പെമാളായി വാഴിച്ചു  അവരുടെ    വാഴിച്ചയും കഴിഞ അവര  പൊയതിന്റെ ശെഷം ബ്രാഹ്മണര വരടെ ശ ത്ത ചെന്ന രായ രുമായി കണ്ട പന്തീരാണ്ടാ  വാഴുവാൻ തക്കവണ്ണം ചെരമാൻ പെ രുമാളെ ന്നരാ ജാ വിനെയും കൽപ്പിച്ചു ചെരമാൻ പെരുമാളും ബ്രാഹ്മണ രും കൂടി പോരു മ്പൊൾ